# Explain

* Check optimized values

* optimization with parameters
*              min, max, smin, smax
opt_amp      = [1, 5]

opt_hcrit    = [np.log10(0.001), np.log10(1)]

opt_rcrit    = [0, 1]

opt_mcrit    = [np.log10(1e-9), np.log10(1e-7)]

opt_alb_smax = [0.7, 0.95]

opt_alb_smin = [0.2, 0.6]

opt_albi     = [0.2, 0.6]


In [ ]:
def unwrap_part(part):
    '''unwarp information given particle
    '''
    if 'amp' in list(part):
        print(f"Tamp:     {part['amp']:.4f}")
    else:
        print(f"Tamp:     {part['amp0']:.4f}")
    print(f"hcrit:    {10**part['hcrit']:.4f}")
    print(f"rcrit:    {part['rcrit']:.4f}")
    print(f"mcrit:    {10**part['mcrit']:.2e}")
    print(f"alb_smax: {part['alb_smax']:.4f}")
    print(f"alb_smin: {part['alb_smin']:.4f}")
    print(f"albi:     {part['albi']:.4f}")

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from issmmodules import *
import matplotlib.pyplot as plt
import pandas
import json
import numpy as np
import scipy.io
from loadmodel_netcdf import *
import pyseb
import tqdm
sys.path.append('./src')
from prepare_semic_day import *

# Summary: PSO-mon-opt_method=2

In [ ]:
# datadir = './data/PSO_mon2_debug/'
datadir = './data/PSO_mon2'
dflog = pandas.read_csv(datadir + 'PSO_summary.csv',index_col=0)

display(dflog)

fig, ax = plt.subplots()
# ax.plot(dflog.index, dflog['avg'], color='b')
# ax.plot(dflog.index, dflog['avg'], color='k', label='min')
ax.errorbar(dflog.index, dflog['avg'], yerr=dflog['std'])
ax.legend()

# now, find where is the last generation?
g = dflog['gen'].values[-1]
Tamp = []
alb_smax = []
hcrit = []
for _g in range(g+1):
    with open(f'{datadir}/PSO_gen{_g}.json','r') as fid:
        part = json.load(fid)
    
    print(f'Current generation: {_g}')
    # unwrap_part(part['0']['global_best'][0])
    Tamp.append(part['0']['global_best']['amp0'])
    alb_smax.append(part['0']['global_best']['alb_smax'])
    hcrit.append(part['0']['global_best']['hcrit'])

# okay, show best values
with open(f'{datadir}/PSO_gen{g}.json','r') as fid:
    part = json.load(fid)
unwrap_part(part['0']['global_best'])

fig, axs = plt.subplots(2,2)
axs = axs.flatten()

ax = axs[0]
ax.plot(dflog.index, np.array(Tamp))
ax.set_xlabel('generation')
ax.set_ylabel('Tamp (K)')

ax = axs[1]
ax.plot(dflog.index, np.array(alb_smax))
ax.set_xlabel('generation')
ax.set_ylabel('maximum snow albedo (-)')
fig.tight_layout()

ax = axs[2]
ax.plot(dflog.index, 10**np.array(hcrit))
ax.set_xlabel('generation')
ax.set_ylabel('critical height (m)')
fig.tight_layout()

# Load ANT model

In [ ]:
from interpRACMO23p2_ERA5 import *
md = loadmodel_netcdf('../data/ANT_Mesh.nc')

In [ ]:
racmo_melt, racmo_time = interpRACMO23p2_ERA5(md.mesh.lat, md.mesh.long, 'snowmelt',
                          'time',[datetime.datetime(1980,1,1), datetime.datetime(2001,1,1)],'use_cftime',1)
racmo_TotalMelt = TransientTotalTimeSeries(md.mesh.elements-1, md.mesh.x, md.mesh.y, racmo_melt,
                                          'mask',md.mask.ice_levelset<0)
racmo_time = cftime2datetime(racmo_time)

# Load ERA5 dataset

In [ ]:
if 0: # interpoalte! dataset
    data_era5 = {}
    for varname in ['d2m','t2m','msr','mtpr','msdwlwrf','msdwswrf','sp','wind2m']:
        print(f'Processing... {varname}')
        ds = interpCDO(md.mesh.lat, md.mesh.long,
                       f'../data/ERA5/day/era5_{varname}_1980,2000.nc', varname=varname)
        data_era5[varname] = ds[varname].values
    
    print('Save dataset in matlab format.')
    scipy.io.savemat('../data/Prepare/ANT_InterpERA5.mat',data_era5)
else:
    print('Load saved dataset.')
    data_era5 = scipy.io.loadmat('../data/Prepare/ANT_InterpERA5.mat')

# Initialize forcing variables of ERA5 for SEMIC

In [ ]:
rho_freshwater = 1000 # kg m-3

# extract ice shelf region
# pos_iceshelf = np.where(md.mask.ocean_levelset < 0)[0]
# pos_iceshelf = np.where(np.ones((md.mesh.numberofvertices,)))[0]
pos_iceshelf = md.mesh.extractedvertices-1

print('ERA5: Set forcing variable for SEMIC.')
sf = data_era5['msr'].T/rho_freshwater
rf = (data_era5['mtpr'].T - data_era5['msr'].T)/rho_freshwater

t2m = data_era5['t2m'].T
sp  = data_era5['sp'].T # surface pressure
lwd = data_era5['msdwlwrf'].T
swd = data_era5['msdwswrf'].T

wind = data_era5['wind2m'].T

# estimate the air density and specific humidity
print('ERA5: Prepare specific humidity and air density')
qs = pyseb.utils.Dewpoint2SpecificHumidity(data_era5['d2m'], data_era5['sp'])
rho_air = pyseb.utils.AirDensityFromSpecificHumidity(data_era5['sp'], data_era5['t2m'], q=qs)
qq = qs.T
rhoa = rho_air.T

nx    = np.shape(qq)[0]
ntime = np.shape(qq)[1]

print(f'ERA5: Available maximum size of time: {ntime}')

In [ ]:
# get default semic
semic = prepare_semic_day()

nloop = 10
nx    = np.shape(sf)[0]
ntime = np.shape(sf)[1]

# initialize results array
# ntime = 365
smb     = np.zeros((nx,ntime))
# smb_snow= np.zeros((nx,ntime))
melt    = np.zeros((nx,ntime))
tsurf   = np.zeros((nx,ntime))
alb     = np.zeros((nx,ntime))
netswd  = np.zeros((nx,ntime))
shf     = np.zeros((nx, ntime))
lhf     = np.zeros((nx, ntime))

for loop in range(nloop):
    print(f'   nloop: {loop}')
    for i in tqdm.tqdm(range(ntime)):
        semic.sf   = sf[:,i]
        semic.rf   = rf[:,i]
        semic.swd  = swd[:,i]
        semic.lwd  = lwd[:,i]
        semic.wind = wind[:,i]
        semic.sp   = sp[:,i]
        semic.rhoa = rhoa[:,i]
        semic.qq   = qq[:,i]
        semic.t2m  = t2m[:,i]
        semic.RunEnergyAndMassBalance()
        # break
        if loop == nloop-1:
            smb[:,i]   = semic.smb
            melt[:,i]  = semic.melt
            tsurf[:,i] = semic.tsurf
            alb[:,i]   = semic.alb
            netswd[:,i]= (1-alb[:,i])*swd[:,i]

unit for each variable

smb ~ water m/sec

In [ ]:
TotalSmb  = TransientTotalTimeSeries(md.mesh.elements-1, md.mesh.x, md.mesh.y ,smb, 'mask',md.mask.ice_levelset<0) *md.materials.rho_freshwater*md.constants.yts
TotalMelt = TransientTotalTimeSeries(md.mesh.elements-1, md.mesh.x, md.mesh.y, melt,'mask',md.mask.ice_levelset<0)*md.materials.rho_freshwater*md.constants.yts

semic_time = pandas.date_range(datetime.datetime(1980,1,1), periods=ntime)
dssemic = xarray.Dataset(data_vars={'smb':(['nx','time'], smb),
                                   'melt':(['nx','time'], melt),
                                   'swsn':(['nx','time'], netswd),
                                   'tsurf':(['nx','time'], tsurf),
                                   'TotalSmb':(['time'], TotalSmb),
                                   'TotalMelt':(['time'], TotalMelt)},
                        coords={'x':('nx',md.mesh.x),
                               'y':('nx',md.mesh.y),
                               'time':('time',semic_time)})
dssemic = dssemic.resample(time="1MS").mean(dim='time')

In [ ]:
display(dssemic)

In [ ]:
plotmodel(md,'nlines',1,'ncols',2,
          'data', dssemic['melt'].mean(axis=0)*md.materials.rho_freshwater/md.materials.rho_ice*md.constants.yts,
         'data',np.mean(racmo_melt,axis=1),
         'caxis#all',[0, 0.1],)

In [ ]:
fig, ax = plt.subplots()
ax.plot(dssemic.time, dssemic.TotalMelt/1e+12)
ax.plot(racmo_time, racmo_TotalMelt*md.materials.rho_ice/1e+12)